# ASSIGNMENT 1

## Lara Monteserín Placer

## María Ferrero Medina

## Introduction

The aim of this project is to design a machine learning model that is able to predict the energy produced by the Sotavento wind farm. For this purpose, a dataset with 555 features ans 4748 instances is available.

The structure of the study will be the following:

1. Exploratory Data Analysis
2. Methodology
3. KNN model
4. Decission tree model
5. Ensemble method 1 model
6. Ensemble method 2 model
7. Selection and performance of the final model

For each of the models created, several steps have been followed to optimize them. First of all, a simple version of each model is created, with hyperparameters that seem reasonable, no feature selection and a basic imputation technique. Then, sequentially, models are improved.

1. First model
2. Feature selection
3. Imputation techniques
4. Hyperparameter tuning


Things to add: - another idea (new library?, new idea...)


## 1. Exploratory Data Analysis

Before starting to build the model, an EDA is made as a first approach to gain understanding of the dataset. In this Exploratory Data Analysis the data type of the features will be verified, the number of instances and features will be determined. Also, a brief summary of the missing values and columns with constant value will be included. 

### 1.1. Number of instances and features

This dataset has 4748 instances and 555 features.

### 1.2. Nature of the variables

This dataset contains information about the meteorological conditions in several locations, the time the measures of these conditions were made and the energy produced at each moment. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Read the data that is compressed as a gzip
wind_ava = pd.read_csv('wind_available.csv.gzip', compression="gzip")

# Display the first rows of the dataset just to see it
wind_ava.head()

# Display the data type of each column
column_data_types = wind_ava.dtypes
print(column_data_types)

energy     float64
year         int64
month        int64
day          int64
hour         int64
            ...   
v100.21    float64
v100.22    float64
v100.23    float64
v100.24    float64
v100.25    float64
Length: 555, dtype: object


After having checked the data types of all the different features, it has been verified that there are:

- 551 numerical variables (real numbers). From this 551, one is the energy, that is the output of the problem. And the remaining 550 are relative to the 22 different meteorological conditions measured at the 25 different locations.

- 4 numerical variables (integers). These 4 variables are the year, day, month and hour of the day. These variables characterize the moment the measures were taken.

### 1.3. Check for missing values 

In [2]:
# Return the number of Null values for each column
null_values = wind_ava.isnull().sum()
# Return the number of NaN values for each column (just in case they are not the same)
nan_values = wind_ava.isna().sum()

# Store in missing values the amount of Null and NaN values of each column
missing_values = pd.DataFrame({
    'Column': null_values.index,
    'Null Values': null_values.values,
    'NaN Values': nan_values.values
})

# Print the amount of Null and Nan values
print(missing_values)

# Identify columns with Null or NaN values
columns_with_null = wind_ava.columns[wind_ava.isnull().any()]
columns_with_nan = wind_ava.columns[wind_ava.isna().any()]

# Display the number of columns which have missing values
print("Number of columns with Null Values:", len(columns_with_null))
print("Number of columns with NaN Values:", len(columns_with_nan))

      Column  Null Values  NaN Values
0     energy            0           0
1       year            0           0
2      month            0           0
3        day            0           0
4       hour            0           0
..       ...          ...         ...
550  v100.21          261         261
551  v100.22          387         387
552  v100.23          569         569
553  v100.24          579         579
554  v100.25          436         436

[555 rows x 3 columns]
Number of columns with Null Values: 550
Number of columns with NaN Values: 550


All meteorological variables have missing values in different instances. The 4 categories that characterize the moment the measure was made and the target feature'energy' do not have missing values.


### 1.4. Check for constant columns


In [3]:
# Check for constant values in each column
constant_columns = wind_ava.columns[wind_ava.nunique() == 1]

# Print columns with constant values
print("Columns with constant values:", constant_columns)

Columns with constant values: Index([], dtype='object')



There are no constant columns. 


### 1.5. Type of problem

The objective of the model is to estimate the energy, as it is a continuous numerical value, this is a **regression problem**. 

## 2. Methodology

This section will explain the methodology that is going to be followed to evaluate the models. The evaluation techniques that will be used for outer evaluation and inner evaluation. And also the metrics that will determine the future performance of the model.
The structure of the assignment will be to first optimize four different models following different strategies. And then compare the best model obtained for each of these four cases and choose the best performance model that has been reached.
The creation and optimmization of each model has been structured as follows: 
- First, a simple approach of the models with KNN as imputation technique, no feature selection and default hyperparameters is created. This aims to obtain a first approach mod
- Second, different feature selection strategies will be made, by hand. Three cases are considered and the best alterntive in terms of the minimum error, is chosen. The goal when performing feature selection first, is to reduce the dimensionality of the data in the following stages.
- Third, different imputation techniques will be compared and then, again, the best strategy in terms of the error will be chosen. Here, the idea is to have the imputation technique that works better for the model before starting the optimization of the hyperparameters.
- Finaly, to obtain the final version of each model, hyperparameter tuning is done. Three different strategies are used for hyperparameter tuning, this is in order to compare the execution times of each of them. But the error will be the strategy to determine later which combination of hyperparameters is the best one.
All these stages will be carried out using only the training set, then, to estimate the future performance, 

- On the one hand, for the **outer evaluation**, holdout evaluation will be used. Thie method will be used to estimate the future performance of the designed method. So, a train set will be used for all the stages of model building and then a test set will be kept to estimate the future performance in an unbiased manner. 

- On the other hand, for the **inner evaluation**, crossvalidation will be the method applied. Crossvalidation will be used to determine which is the best combination of hyperparameters. Also, to determine the best strategy for feature selection and for imputation techniques, crossvalidation will be used, using only the available data in the training set. 

Later, to improve the performance of the method, once it is already computed the outer performance, the hyperparameters will be tuned again but this time using the whole training set. And finaly, the model will be trained using the whole training dataset. 

The objective function that is going to be used for the validation of the method, and for comparing the methods between them is the Root Mean Squared Error (RMSE). This metric is more sensitive to outliers and to distant values than the MAE as it squares the magnitudes. This is useful to penalize the errors that are larger, and avoid having a model that might have such large errors. RMSE has been used instead of MSE as it is easier to interpret considering it has the same order of magnitude than the target feature (the energy in this case).

Once this study has been done, the models will be trained using the complete dataset to develop the final model. This final model will be then used with the new data that is provided in a separate dataset. 

**It could be longer this explanation**

Note: as the variable that is going to be predicted is the 'energy', only the variables related to the meteorological characteristics will be used. It is considered that the energy produced does not depend on the moment of the day it is being produced.


The train and test split that is done for holdout validation is made considering that the data comes 

In [4]:
#Here, we define a generic random seed that will be used along the assignment
rs = 100515585 

# First, data will be divided into train and test set 
# Considering it is a time series, it must be split in an appropiate way
wind_ava['timestamp'] = pd.to_datetime(wind_ava[['year', 'month', 'day', 'hour']])
wind_ava = wind_ava.sort_values(by='timestamp')

train_size = 0.8 
split_index = int(len(wind_ava) * train_size)
wind_ava = wind_ava.drop(columns=['timestamp'])

# Divide the data into X_train, y_train, X_test, y_test
train_data = wind_ava.iloc[:split_index]
test_data = wind_ava.iloc[split_index:]

X_train = train_data.drop('energy', axis=1)
y_train = train_data['energy']
X_test = test_data.drop('energy', axis=1)
y_test = test_data['energy']

# For inner validation for feature selection and imputation
# Divide the train set into inner train set (for training) and inner validation set (for validation)

train_size_inner = 0.8
split_index_inner = int(len(train_data) * train_size_inner)

# Divide the data Inner Train Set and Inner Validation Set
train_inner = train_data.iloc[:split_index_inner]
val_inner = train_data.iloc[split_index_inner:]


## 3. KNN Regressor

The algorithm to try is the KNN algorithm applied for Regression. In this section, different imputation techniques and feature selection strategies will  be used to determine which is the most adequate. Later, hyperparameter tuning will determine the optimal number of neighbors to be used. Finally, a final version of the model will be built taking all these studies into consideration. 

### 3.1. First model

A first model using the KNN algorithm will be implemented. It will use the default hyperparameters, the KNN Imputer as imputation strategy and no feature selection. This model will be updated and improved later but a first approach is required to have a model to compare in further stages.

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
import numpy as np

# Now the first KNN model will be created
first_knn = Pipeline([('imputer',KNNImputer()),('regression',KNeighborsRegressor())])
first_knn.fit(X_inner_train,y_inner_train)
y_predicted_first = first_knn.predict(X_inner_val)
MSE = mean_squared_error(y_inner_val,y_predicted_first)

print('RMSE for the first KNN model: ', np.sqrt(MSE))
# RMSE: 650.97

RMSE for the first KNN model:  650.972879219549


### 3.2 Feature selection

Now, feature selection will be used in order to remove variables that are not affecting the target and thus avoid overfitting that may be lead by wrong interpretation from these variables. This has been done manually, selecting different feature combinations that seem to be accurate for the problem. Three different cases have been considered:
- FIRST OPTION. Selecting only the features related to the location of the wind farm (Sotavento). This is the features that contain the sufix 13.
- SECOND OPTION. Selecting only the features related to the wind characteristics. This is the features that start with u or v (the vertical and horizontal components of the wind).
- THIRD OPTION. Selecting only the features which are both located in Sotavento (location 13) and related to the wind characteristics. This is the features that start with u or v and end with sufix 13.

This strategy will be followed for the feature selection of the rest of the models. This is sections 4.2, 5.2 and 6.2.

In [6]:
# FIRST OPTION - Selecting only the features that correspond to the location 13 (Sotavento)

# Select the desired variables
X_inner_train_1 = train_inner.filter(regex='\.13$', axis=1)
X_inner_val_1 = val_inner.filter(regex='\.13$', axis=1)

# The first model that has been already created is trained now with the selected data
first_knn.fit(X_inner_train_1,y_inner_train)
y_predicted_1 = first_knn.predict(X_inner_val_1)
MSE_1 = mean_squared_error(y_inner_val,y_predicted_1)

print('RMSE for feature selection with variables from Sotavento: ', np.sqrt(MSE_1))
# 658.28

RMSE for feature selection with variables from Sotavento:  658.2887149048381


In [7]:
# SECOND OPTION - Selecting only the features related to the wind (the ones that start with u or v)

# Select the desired variables
X_inner_train_2 = train_inner.filter(regex='^(u|v).*$', axis=1)
X_inner_val_2 = val_inner.filter(regex='^(u|v).*$', axis=1)

# The first model that has been already created is trained now with the selected data
first_knn.fit(X_inner_train_2,y_inner_train)
y_predicted_2 = first_knn.predict(X_inner_val_2)
MSE_2 = mean_squared_error(y_inner_val,y_predicted_2)

print('RMSE for imputation with variables related to the wind: ', np.sqrt(MSE_2))
# 428.12

RMSE for imputation with variables related to the wind:  428.12271404149146


In [8]:
# THIRD OPTION - Selecting the features that correspond to magnitudes related to the wind in Sotavento

# Select the desired variables
X_inner_train_3 = train_inner.filter(regex='^(u|v).*\.13$', axis=1)
X_inner_val_3 = val_inner.filter(regex='^(u|v).*\.13$', axis=1)

# The first model that has been already created is trained now with the selected data
first_knn.fit(X_inner_train_3,y_inner_train)
y_predicted_3 = first_knn.predict(X_inner_val_3)
MSE_3 = mean_squared_error(y_inner_val,y_predicted_3)

print('RMSE for imputation with variables from Sotavento related to the wind: ', np.sqrt(MSE_3))
# 433.40

RMSE for imputation with variables from Sotavento related to the wind:  433.4095713141627


The results from different feature selection strategies are included in the following table. These results show that the best feature selection strategy for the model is the second, selecting only the features related to the wind. Feature selection improves the performance of the model as it discards features that are not affecting the target and may lead to overfitting. 

|          | 0. No feature selection | 1. Sotavento features | 2. Wind features | 3. Sotavento wind features |
|-----------|:----------------------:|:---------------------:|:----------:|:----------:|
| RMSE | 658.28 | 663.35 | 428.12 | 433.40 |


From this moment, the KNN model will be built using only the features related to the wind, that is the variables categorized as X_train_inner_2, X_val_inner_2.

### 3.3. Imputation techniques

The dataset includes many missing values, it is important to apply an adequate imputation technique. In this section, three different imputation techniques will be considered and applied. The results obtained from this study will determine which is the imputation technique that will be followed later. Three different strategies have been studied:
- FIRST OPTION: Simple Imputer. It is an univariate imputation technique, this means that only uses values relative to the feature that is going to be imputed. It imputes the missing values with the mean of the feature. The mean is been chosen instead of the median because there are not many outliers that could affect the distribution of the data.
- SECOND OPTION: KNN Imputer. It is a multivariate imputation technique, this means that it uses values from the feature that is going to be imputed but also from other features. This technique in partiular is based in the KNN algorithm and it consistsn on imputing the value os the missing category as a mean from the closest neighbors.
- THIRD OPTION: Iterative Imputer. It is also a muktivarite technique. Is based on iterative models that compute the values for the missing categories.


In [12]:
# FIRST OPTION: Simple Imputer using the mean

from sklearn.impute import SimpleImputer
simple_knn = Pipeline([('imputer',SimpleImputer(strategy = 'mean')),('regression',KNeighborsRegressor())])

simple_knn.fit(X_inner_train_2,y_inner_train)
y_predicted_simple_imputer = simple_knn.predict(X_inner_val_2)
MSE_simple = mean_squared_error(y_inner_val,y_predicted_simple_imputer)

print('RMSE for imputation with Simple Imputer: ', np.sqrt(MSE_simple))
# 457.67

RMSE for imputation with Simple Imputer:  457.67268509394927


In [13]:
# SECOND OPTION: KNN Imputer with default hyperparameters

from sklearn.impute import KNNImputer
knn_knn = Pipeline([('imputer',KNNImputer()),('regression',KNeighborsRegressor())])

knn_knn.fit(X_inner_train_2,y_inner_train)
y_predicted_knn_imputer = knn_knn.predict(X_inner_val_2)
MSE_knn = mean_squared_error(y_inner_val,y_predicted_knn_imputer)

print('RMSE for imputation with KNN Imputer: ', np.sqrt(MSE_knn))
# 428.12

RMSE for imputation with KNN Imputer:  428.12271404149146


In [ ]:
# Update the version of scikit-learn if it returns an Error with IterativeImputer
!pip install --upgrade scikit-learn --user

In [14]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# THIRD OPTION: multivariate technique. Iterative imputer with 10 maximum iterations. Include random seed for reproducibility
iterative_knn = Pipeline([('imputer',IterativeImputer(max_iter = 10, random_state = rs)),('regression',KNeighborsRegressor())])

iterative_knn.fit(X_inner_train_2,y_inner_train)
y_predicted_iterative_imputer = iterative_knn.predict(X_inner_val_2)
MSE_iterative = mean_squared_error(y_inner_val,y_predicted_iterative_imputer)

print('RMSE for imputation with Iterative Imputer: ', np.sqrt(MSE_iterative))
# 426.14

RMSE for imputation with Iterative Imputer:  426.13631965182225


|          | 1. Simple Imputer | 2. KNN Imputer | 3. Iterative Imputer|
|-----------|:----------------------:|:---------------------:|:----------:|
| RMSE | 457.67 | 428.12 | 426.14 |

From the results above, it is demonstrated that the Simple Imputer is the worst in terms of performance with respect to the Mean Squared Error.

On the other hand, the multivariate techniques, KNN Imputer and the Iterative Imputer give similar results. The Iterative Imputer gets a lower Mean Squared Error, so it will be the chosen technique. 

### 3.4. Hyperparameter tuning

Once the preprocessing stages have been optimized and improved, it is time to optimize the model by performing hyperparameter tuning. Crossvalidation will be the stratefy for HPO. In the case of KNN algorithm, the main hyperparameter is the number of neighbors, this number of neighbors. 

Several approaches for hyperparameter tuning will be implemented. The optimization time will be measured and also the best combination of hyperparameters for each method will be compared. This additional study will provide insight in which HPO strategy is the most adequate in terms of execution time and also in terms of performance.

For HPO, Pipelines will not be used, the preprocessing stages will be made previously in order to make the process faster.

In [84]:
# PREPROCESSING

# FILTER. Feature selection
X_train_filtered = X_train.filter(regex='^(u|v).*$', axis=1)

# IMPUTER. Imputation of missing values
imputer = IterativeImputer(max_iter=10, random_state=rs)
X_train_imputed = imputer.fit_transform(X_train_filtered)
# X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train_filtered), columns=X_train_filtered.columns)


# Preprocessing also for the test data for estimation of future performance later
X_test_filtered = X_test.filter(regex='\.13$', axis=1)
X_test_imputed = imputer.fit_transform(X_test_filtered)
# X_test_imputed_df = pd.DataFrame(X_test_imputed, columns=X_test.columns)


In [85]:
# CREATE THE MODEL TO OPTIMIZE

# Create the regressor with no hyperparamters defined
knn_default = KNeighborsRegressor()

# Create a dictionary with the values for the hyperparameters
params_knn = {'n_neighbors': np.arange(1, 51)}

In [86]:
# FIRST STRATEGY - Randomized Search

# Import required libraries
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
import time

# Create a randomized search instance
random_search = RandomizedSearchCV(
    knn_default,
    param_distributions=params_knn,
    n_iter=10,  # Number of iterations (can be adjusted)
    cv=5,       # Number of cross-validation folds
    scoring='neg_mean_squared_error',  # Mean squared error is the error metric
    random_state=rs
)

# Train the model, optimize the hyperparameters and measure the time
t_0 = time.time()
random_search.fit(X_train_imputed, y_train)
t_1 = time.time()

# Calculate the crossvalidation score for this model
best_params_random = random_search.best_params_
best_knn_random = KNeighborsRegressor(**best_params_random) 
cv_scores = cross_val_score(best_knn_random, X_train_imputed, y_train, cv=5, scoring='neg_mean_squared_error')
mean_cv_rmse = np.mean(np.sqrt(-cv_scores))

# Print results
print("RANDOM SEARCH")
print("Best Hyperparameters for KNN:", best_params_random)
print("Execution time: ",t_1-t_0)
print("CV MSE Score: ", mean_cv_rmse)


best_params {'n_neighbors': 17}
RANDOM SEARCH
Best Hyperparameters for KNN: {'n_neighbors': 17}
Execution time:  1.3825676441192627
CV MSE Score:  391.7182984002424


In [87]:
# THIS STAGE TAKES LONG
# SECOND STRATEGY - Optuna

# Import required libraries
import optuna
from sklearn.model_selection import cross_val_score, KFold

# Define the objective function
def objective(trial):
    # Hyperparameters that are to be tuned
    n_neighbors = trial.suggest_int('n_neighbors', 1, 51)
    params = {'n_neighbors': n_neighbors}
    
    # Estimator with suggested hyperparameters
    knn_default = KNeighborsRegressor()
    
    # Define the neg means squared error as the score and the inner evaluation as corssvalidation 
    inner_score = cross_val_score(iterative_knn, X_train_imputed, y_train, cv=5, scoring='neg_mean_squared_error').mean()
    return inner_score

# Train the model and perform HPO
# Measure the time
sampler = optuna.samplers.TPESampler(seed=rs)
study_knn = optuna.create_study(direction='maximize',sampler=sampler)

iterations = 30
t_0 = time.time()
study_knn.optimize(objective, n_trials = iterations)
t_1 = time.time()

# Best hyperparameters
best_params_optuna = study_knn.best_params

# Calculate the crossvalidation score for this model
best_knn_optuna = KNeighborsRegressor(**best_params_optuna) 
cv_scores_optuna = cross_val_score(best_knn_optuna, X_train_imputed, y_train, cv=5, scoring='neg_mean_squared_error')
mean_cv_rmse_optuna = np.mean(np.sqrt(-cv_scores_optuna))

# Print results
print("OPTUNA HPO")
print("Best Hyperparameters for KNN:", best_params_optuna)
print("Execution time: ",t_1-t_0)
print("CV RMSE Score: ", mean_cv_rmse_optuna)

[I 2024-01-02 20:25:24,937] A new study created in memory with name: no-name-af39b7fe-e670-473c-adfb-839e51849685
[I 2024-01-02 20:25:42,513] Trial 0 finished with value: -170261.21944310627 and parameters: {'n_neighbors': 21}. Best is trial 0 with value: -170261.21944310627.
[I 2024-01-02 20:26:00,560] Trial 1 finished with value: -170261.21944310627 and parameters: {'n_neighbors': 33}. Best is trial 0 with value: -170261.21944310627.
[I 2024-01-02 20:26:18,122] Trial 2 finished with value: -170261.21944310627 and parameters: {'n_neighbors': 50}. Best is trial 0 with value: -170261.21944310627.
[I 2024-01-02 20:26:35,977] Trial 3 finished with value: -170261.21944310627 and parameters: {'n_neighbors': 40}. Best is trial 0 with value: -170261.21944310627.
[I 2024-01-02 20:26:54,141] Trial 4 finished with value: -170261.21944310627 and parameters: {'n_neighbors': 37}. Best is trial 0 with value: -170261.21944310627.
[I 2024-01-02 20:27:11,860] Trial 5 finished with value: -170261.219443

OPTUNA HPO
Best Hyperparameters for KNN: {'n_neighbors': 21}
Execution time:  532.7751214504242
CV RMSE Score:  391.52835078116124


In [92]:
# THIRD STRATEGY - Successive halving

# Import required libraries
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV


# Create
halving = HalvingGridSearchCV(knn_default,
                              params_knn,
                              scoring= 'neg_mean_squared_error',
                              cv=5,
                              random_state=rs,
                              factor=2,
                              min_resources='exhaust',
                              max_resources='auto',
                              n_jobs=-1, verbose=1)

# Train the model, optimize the hyperparameters and measure the time
t_0 = time.time()
halving.fit(X_train_imputed, y_train)
t_1 = time.time()

# Calculate the crossvalidation score for this model
best_params_halving = halving.best_params_
best_knn_halving = KNeighborsRegressor(**best_params_halving) 
cv_scores_halving = cross_val_score(best_knn_halving, X_train_imputed, y_train, cv=5, scoring='neg_mean_squared_error')
mean_cv_rmse_halving = np.mean(np.sqrt(-cv_scores_halving))

# Print results
print("SUCCESIVE HALVING")
print("Best Hyperparameters for KNN:", best_params_halving)
print("Execution time: ",t_1-t_0)
print("CV MSE Score: ", mean_cv_rmse_halving)

n_iterations: 6
n_required_iterations: 6
n_possible_iterations: 6
min_resources_: 118
max_resources_: 3798
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 50
n_resources: 118
Fitting 5 folds for each of 50 candidates, totalling 250 fits
----------
iter: 1
n_candidates: 25
n_resources: 236
Fitting 5 folds for each of 25 candidates, totalling 125 fits
----------
iter: 2
n_candidates: 13
n_resources: 472
Fitting 5 folds for each of 13 candidates, totalling 65 fits
----------
iter: 3
n_candidates: 7
n_resources: 944
Fitting 5 folds for each of 7 candidates, totalling 35 fits
----------
iter: 4
n_candidates: 4
n_resources: 1888
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 5
n_candidates: 2
n_resources: 3776
Fitting 5 folds for each of 2 candidates, totalling 10 fits
SUCCESIVE HALVING
Best Hyperparameters for KNN: {'n_neighbors': 11}
Execution time:  1.9224305152893066
CV MSE Score:  393.42255429468116


|          | 1. Random Search | 2. OPTUNA | 3. Successive Halving |
|-----------|:----------------------:|:---------------------:|:----------:|
| RMSE (crossvalidation) | 391.71 | 391.52 | 393.42 |
| n_neighbors | 17 | 21 | 11 |
| Time (s) | 1.38 | 532.77 | 1.92 |

The results from the HPO show that the number of neighbors does not seem to affect the performance when it changes from 17 to 21, as the crossvalidation scores are the same. But, it also shows that the  

### 3.5. Conclusions about KNN model

Out of all the trials, the KNN model has demonstrated to work better with the following adjustments:
- Best number of neighbors:
- Best imputation technique: Iterative Imputer
- Best feature selection strategy: selecting the variables related to wind characteristics.
Now, to estimate the future performance of this model, the test set that has been previously set aside will be used.

In [91]:
from sklearn.preprocessing import FunctionTransformer

# Define optimal hyperparameters
opt_k = 17

# Custom function for filtering
def filter_knn(data):
    return data.filter(regex='^(u|v).*$', axis=1)

# Define the pipeline
best_knn = Pipeline([
    ('filter', FunctionTransformer(filter_knn, validate=False)),
    ('imputer',IterativeImputer(max_iter = 10, random_state = rs)),
    ('regression',KNeighborsRegressor(n_neighbors = opt_k))])

# Fit training data
best_knn.fit(X_train,y_train)

# Predict test data
y_pred_best = best_knn.predict(X_test)
MSE_best_knn = mean_squared_error(y_test,y_pred_best)

print('RMSE for the optimized KNN model: ', np.sqrt(MSE_best_knn))

RMSE for the optimized KNN model:  407.98965946652226


### 3.6. Training the KNN model with the whole training set

Now that feature selection, imputation and hyperparameters have been optimized, the final model is trained using the whole wind_ava dataset. A pipeline with all the optimized stages will be done.


## 4. Decision Tree

### 4.1. First model

A first model using the Decision Tree algorithm will be implemented. It will use the default hyperparameters, the KNN Imputer as imputation strategy and no feature selection. This model will be updated and improved but it is just a simple approach to later compare the adjusted models. 

In [61]:
from sklearn.tree import DecisionTreeRegressor

# Create the first Decission Tree model
first_tree = Pipeline([('imputer',KNNImputer()),('regression',DecisionTreeRegressor(random_state=rs))])
first_tree.fit(X_inner_train,y_inner_train)
y_predicted = first_tree.predict(X_inner_val)
MSE_tree = mean_squared_error(y_inner_val,y_predicted_first)

print('RMSE for the first KNN model: ', np.sqrt(MSE_tree))
# 650.97

RMSE for the first KNN model:  650.972879219549


### 4.2. Feature selection

Same procedure as in section 3.2. but in this case using Decission Tree Regressor model. The variable selection will not be made again in this section, the filtered datasets are defined in section 3.2.

In [62]:
# FIRST OPTION - Selecting only the features that correspond to the location 13 (Sotavento)

# The first Decission Tree model that has been already created is trained now with the selected data
first_tree.fit(X_inner_train_1,y_inner_train)
y_predicted_1 = first_tree.predict(X_inner_val_1)
MSE_1 = mean_squared_error(y_inner_val,y_predicted_1)

print('RMSE for feature selection with variables from Sotavento: ', np.sqrt(MSE_1))
# 522.43

RMSE for feature selection with variables from Sotavento:  522.4285433416003


In [63]:
# SECOND OPTION - Selecting only the features related to the wind (the ones that start with u or v)

# The first Decission Tree model that has been already created is trained now with the selected data
first_tree.fit(X_inner_train_2,y_inner_train)
y_predicted_2 = first_tree.predict(X_inner_val_2)
MSE_2 = mean_squared_error(y_inner_val,y_predicted_2)

print('RMSE for feature selection with variables related to the wind: ', np.sqrt(MSE_2))
# 541.63

RMSE for feature selection with variables from Sotavento:  541.6336936084882


In [64]:
# THIRD OPTION - Selecting the features that correspond to magnitudes related to the wind in Sotavento

# The first model that has been already created is trained now with the selected data
first_tree.fit(X_inner_train_3,y_inner_train)
y_predicted_3 = first_tree.predict(X_inner_val_3)
MSE_3 = mean_squared_error(y_inner_val,y_predicted_3)

print('RMSE for imputation with variables from Sotavento related to the wind: ', np.sqrt(MSE_3))
# 544.00

RMSE for imputation with variables from Sotavento related to the wind:  543.9981445138152


The results from different feature selection strategies are included in the following table. These results show that the feature selection strategy that works better with Decission Trees is the first strategy, this is, choosing the variables related to Sotavneto only.

|          | 0. No feature selection | 1. Sotavento features | 2. Wind features | 2. Wind features in Sotavento |
|-----------|:----------------------:|:---------------------:|:----------:|:----------:|
| RMSE |  650.97 | 522.42 | 541.63 | 543.99 |


From this moment, the KNN model will be built using only the features that were measured in Sotavento.

The best RMSE achieved until this point is: 522.42

### 4.3. Imputation techniques

Same procedure as in section 3.3. but in this case using Decission Tree Regressor model.

In [68]:
# FIRST OPTION: Simple Imputer using the mean
simple_tree = Pipeline([('imputer',SimpleImputer(strategy = 'mean')),('regression',DecisionTreeRegressor(random_state=rs))])

simple_tree.fit(X_inner_train_1,y_inner_train)
y_predicted_simple_imputer = simple_tree.predict(X_inner_val_1)
MSE_simple = mean_squared_error(y_inner_val,y_predicted_simple_imputer)

print('RMSE for imputation with Simple Imputer: ', np.sqrt(MSE_simple))
# 546.31

RMSE for imputation with Simple Imputer:  546.3131100360276


In [70]:
# SECOND OPTION: KNN Imputer with default hyperparameters
knn_tree = Pipeline([('imputer',KNNImputer()),('regression',DecisionTreeRegressor(random_state=rs))])

knn_tree.fit(X_inner_train_1,y_inner_train)
y_predicted_knn_imputer = knn_tree.predict(X_inner_val_1)
MSE_knn = mean_squared_error(y_inner_val,y_predicted_knn_imputer)

print('RMSE for imputation with KNN Imputer: ', np.sqrt(MSE_knn))
# 522.43

RMSE for imputation with KNN Imputer:  522.4285433416003


In [71]:
# THIRD OPTION: Iterative Imputer with 10 maximum iterations. Include random seed for reproducibility
iterative_tree = Pipeline([('imputer',IterativeImputer(max_iter = 10, random_state = rs)),('regression',DecisionTreeRegressor(random_state=rs))])

iterative_tree.fit(X_inner_train_1,y_inner_train)
y_predicted_iterative_imputer = iterative_tree.predict(X_inner_val_1)
MSE_iterative = mean_squared_error(y_inner_val,y_predicted_iterative_imputer)

print('RMSE for imputation with Iterative Imputer: ', np.sqrt(MSE_iterative))
# 527.26

RMSE for imputation with Iterative Imputer:  527.2567305971542


The results from different imputation techniques are included in the following table. 

|          | 1. Simple Imputer | 2. KNN Imputer | 3. Iterative Imputer | 
|-----------|:----------------------:|:---------------------:|:----------:|
| RMSE |  546.31 | 522.43 | 527.26 |

Results from the table show that the best imputation technique for Decission Trees in this case is the KNN Imputer with default hyperparameters. From this point, for the Decission Tree model, KNN Imputer will be used. 

### 4.4. Hyperparameter tuning

Once the imputation and feature selection have been performed, it is time to improve the performance of the model with HPO. Hyperparameter tuning will be carried out using crossvalidation as the inner evaluation method. 

In [94]:
# PREPROCESSING

# FILTER. Feature selection
X_train_filtered_tree = X_train.filter(regex='\.13$', axis=1)

# IMPUTER. Imputation of missing values
imputer_tree = KNNImputer()
X_train_imputed_tree = imputer_tree.fit_transform(X_train_filtered_tree)

# Preprocessing also for the test data for estimation of future performance later
X_test_filtered_tree = X_test.filter(regex='\.13$', axis=1)
X_test_imputed_tree = imputer_tree.fit_transform(X_test_filtered_tree)
# X_test_imputed_df = pd.DataFrame(X_test_imputed, columns=X_test.columns)

In [98]:
# CREATE THE MODEL TO OPTIMIZE
from sklearn.model_selection import KFold

# Create the regressor with no hyperparamters defined
tree_default = DecisionTreeRegressor()

# Create a dictionary with the values for the hyperparameters
# Define the grid with all possible values for the hyperparameters
# Define the number of folds and the conditions for the crossvalidation
params_tree = {'max_depth': list(range(2,16,2)), 'min_samples_split':list(range(2,16,2))}
k_folds = KFold(n_splits = 5, shuffle = True, random_state =rs)


In [101]:
# FIRST STRATEGY - Randomized Search

# Import required libraries
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
import time

# Create a randomized search instance
random_search_tree = RandomizedSearchCV(
    tree_default,
    param_distributions=params_tree,
    n_iter=10,  # Number of iterations (can be adjusted)
    cv=5,       # Number of cross-validation folds
    scoring='neg_mean_squared_error',  # Mean squared error is the error metric
    random_state=rs
)

# Train the model, optimize the hyperparameters and measure the time
t_0 = time.time()
random_search_tree.fit(X_train_imputed_tree, y_train)
t_1 = time.time()

# Calculate the crossvalidation score for this model
best_params_random_tree = random_search_tree.best_params_
best_tree_random = DecisionTreeRegressor(**best_params_random_tree) 
cv_scores_random_tree = cross_val_score(best_tree_random, X_train_imputed_tree, y_train, cv=5, scoring='neg_mean_squared_error')

# Print results
print("RANDOM SEARCH")
print("Best Hyperparameters for Decission Tree:", best_params_random_tree)
print("Execution time: ",t_1-t_0)
print("CV mean MSE Score: ", np.mean(-cv_scores_random_tree))

RANDOM SEARCH
Best Hyperparameters for Decission Tree: {'min_samples_split': 2, 'max_depth': 6}
Execution time:  3.1810808181762695
CV mean MSE Score:  196838.08065346064


In [96]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn import metrics
import time

# FIRST STRATEGY: GridSearch 

# Measure the time it takes to perform the hyperparameter tuning
t_0 = time.time()
# Perform the hyperparameter tuning using the previously defined k_folds for crossvalidation
tuning = GridSearchCV(
    tree_default,
    param_grid,
    scoring='neg_mean_squared_error',
    cv=k_folds,
    n_jobs=1,verbose=1)

# Fit the models (all possible combinations)
tuning.fit(X_train_imputed,y_train_2)
t_1 = time.time()

# Access the best estimator
best_model = tuning.best_estimator_

# Access the best hyperparameters
best_params = tuning.best_params_

# Access the cross-validated results
cv_results = tuning.cv_results_

# Print MSE for each fold
for i in range(5):
    fold_mse = -cv_results[f"split{i}_test_score"][tuning.best_index_]
    print(f"MSE for Fold {i+1}: {fold_mse}")

# Calculate and print the mean MSE across folds
mean_mse = -cv_results["mean_test_score"][tuning.best_index_]


# Print the results for the best combination
print("Mean MSE across Folds: ", mean_mse)
print("Best Hyperparameters: ", best_params)
print("Execution time: ",t_1-t_0)

# ESTIMATE FUTURE PERFORMANCE



NameError: name 'y_train_2' is not defined

In [23]:
# Estimation of future performance
y_test_pred = tuning.predict(X_test_imputed)

print("KNN's MSE with HPO: ",
     mean_squared_error(y_test_2, y_test_pred))


KNN's MSE with HPO:  189639.60576969493


In [ ]:
# Install OPTUNA if required
!pip install optuna

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score, KFold

# SECOND STRATEGY: Optuna

# Define the objective function
def objective(trial):
    # Hyperparameters that are going to be tuned
    max_depth = trial.suggest_int('max_depth',2,16)
    min_samples_split = trial.suggest_int('min_samples_split',2,16)
    
    # Estimator with suggested hyperparameters
    params = {'max_depth': max_depth, 'min_samples_split':min_samples_split}
    regr_tree = DecisionTreeRegressor(random_state = 100515585, **params)
    
    # Define the neg means squared error as the score and the inner evaluation as corssvalidation 
    inner_score = cross_val_score(regr_tree,X_train_2,y_train_2,cv=k_folds,scoring='neg_mean_absolute_error').mean()
    return inner_score

# Train the model and perform HPO
# Measure the time
t_0 = time.time()
sampler = optuna.samplers.TPESampler(seed=rs)
study = optuna.create_study(direction='maximize',sampler=sampler)
iterations = 30
study.optimize(objective, n_trials = iterations)
t_1 = time.time()

# Best hyperparameters
best_params = study.best_params
regr_opt = DecisionTreeRegressor(random_state = rs, **best_params)
regr_opt.fit(X_train_2,y_train_2)

# ESTIMATE FUTURE PERFORMANCE
y_test_pred = regr_opt.predict(X_test_2)
mse_future = mean_squared_error(y_test_pred, y_test_2)

# Print results
# Print the results for the best combination
print("MSE future performance: ", mse_future)
print("Best Hyperparameters: ", best_params)
print("Execution time: ",t_1-t_0)

In [ ]:
# Install optuna if required 
!pip install optuna scikit-optimize

In [ ]:
#Successive Halving



import optuna
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import time

# Define the objective function
def objective(trial):
    max_depth = trial.suggest_int('max_depth', 2, 16)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 16)
    params = {'max_depth': max_depth, 'min_samples_split': min_samples_split}
    regr_tree = DecisionTreeRegressor(random_state=100515585, **params)
    inner_score = cross_val_score(regr_tree, X_train_2, y_train_2, cv=k_folds, scoring='neg_mean_absolute_error').mean()
    return inner_score

if __name__ == "__main__":
    t_0 = time.time()

    # Use TPESampler for Bayesian optimization
    sampler = optuna.samplers.TPESampler(seed=100515585)
    study = optuna.create_study(direction='maximize', sampler=sampler)

    # Run optimization
    n_trials = 30
    for _ in range(n_trials):
        trial = study.ask()
        value = objective(trial)
        study.tell(trial, value)

    t_1 = time.time()

    best_params = study.best_params
    regr_opt = DecisionTreeRegressor(random_state=100515585, **best_params)
    regr_opt.fit(X_train_2, y_train_2)

    y_test_pred = regr_opt.predict(X_test_2)
    mse_future = mean_squared_error(y_test_pred, y_test_2)

    print("MSE future performance: ", mse_future)
    print("Best Hyperparameters: ", best_params)
    print("Execution time: ", t_1 - t_0)


## 5. Random Forest Regressor

### 5.1 First model


In [ ]:
from sklearn.ensemble import RandomForestRegressor
t_0 = time.time()
# Create the ensemble model pipeline
random_forest = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42)) 
])

# Fit the model on the training data
random_forest.fit(X_train, y_train)

# Predict on the test data
y_pred = random_forest.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

### 5.2 Feature selection


In [ ]:
# FIRST OPTION - Selecting only the features that correspond to the location 13 (Sotavento)
first_forest = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42)) 
])

# Fit the model on the training data
first_forest.fit(X_train_1, y_train_1)

# Predict on the test data
y_predicted_1_rf = first_forest.predict(X_test_1)

# Evaluate the model
MSE_1 = mean_squared_error(y_test_1, y_predicted_1_rf)
print("Mean Squared Error:", MSE_1)

In [ ]:
# SECOND OPTION - Selecting only the features related to the wind (the ones that start with u or v)

second_forest = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42)) 
])

# Fit the model on the training data
second_forest.fit(X_train_2,y_train_2)

# Predict on the test data
y_predicted_2_rf = second_forest.predict(X_test_2)

# Evaluate the model
MSE_2 = mean_squared_error(y_test_2, y_predicted_2_rf)
print("Mean Squared Error:", MSE_1)


In [ ]:
# THIRD OPTION - Selecting the features that correspond to magnitudes related to the wind in Sotavento

third_forest = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42)) 
])

# Fit the model on the training data
third_forest.fit(X_train_3,y_train_3)

# Predict on the test data
y_predicted_3_rf = third_forest.predict(X_test_3)

# Evaluate the model
MSE_3 = mean_squared_error(y_test_3, y_predicted_3_rf)
print("Mean Squared Error:", MSE_3)


El error no mejora aplicando feature selection.

## 6. Bagging Regressor with KNN models

The last model to be built is also an ensemble model. In this case it consists of an aggregation of KNN models. 
### 6.1 First model

First of all, a model will be created with the default hyperparameters, no feature selection and the KNN algorithm for imputation. This model will be taken as reference to compare with other models buit under the same technique. 


In [ ]:
from sklearn.ensemble import BaggingRegressor

# Create the ensemble model pipeline
knn_ensemble_pipeline = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=10, random_state=42))  
])

# Fit the model on the training data
knn_ensemble_pipeline.fit(X_train, y_train)

# Predict on the test data
y_pred = knn_ensemble_pipeline.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


### 6.2 Feature selection

In [ ]:
# FIRST OPTION - Selecting only the features that correspond to the location 13 (Sotavento)

first_ensemble = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=10, random_state=42))  
])

# Fit the model on the training data
first_ensemble.fit(X_train_1, y_train_1)

# Predict on the test data
y_predicted_1_e = first_ensemble.predict(X_test_1)

# Evaluate the model
MSE_1 = mean_squared_error(y_test_1, y_predicted_1_e)
print("Mean Squared Error:", MSE_1)


In [ ]:
# SECOND OPTION - Selecting only the features related to the wind (the ones that start with u or v)

second_ensemble = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=10, random_state=42))  
])

# Fit the model on the training data
second_ensemble.fit(X_train_2, y_train_2)

# Predict on the test data
y_predicted_2_e = second_ensemble.predict(X_test_2)

# Evaluate the model
MSE_2 = mean_squared_error(y_test_2, y_predicted_2_e)
print("Mean Squared Error:", MSE_2)

In [ ]:
# THIRD OPTION - Selecting the features that correspond to magnitudes related to the wind in Sotavento

third_ensemble = Pipeline([
    ('imputer',KNNImputer(n_neighbors=3)),
    ('regressor', BaggingRegressor(base_estimator=KNeighborsRegressor(n_neighbors=5), n_estimators=10, random_state=42))  
])

# Fit the model on the training data
third_ensemble.fit(X_train_3, y_train_3)

# Predict on the test data
y_predicted_3_e = third_ensemble.predict(X_test_3)

# Evaluate the model
MSE_3 = mean_squared_error(y_test_3, y_predicted_3_e)
print("Mean Squared Error:", MSE_3)

From the previous results, it is demonstrated that the error from the model improves applying feature selection. The best results are obtained selecting only the features related to the wind. The second best result is obtained selecting only the features from Sotavento and related to the wind.

Based on this results, for the KNN model, only the features related to the wind will be selected (second option). So, from now on, it will be used X_2 and y_2.